# Imports

In [ ]:
import pandas as pd
import string
import numpy as np
import os
import re

# Grab File

In [ ]:
year = "2013"

import_csv = year + "_SSE.csv"

csv = pd.read_csv(import_csv, 
                   sep='\t', 
                   parse_dates=[3])

## Inspect Data

In [ ]:
csv.head()

# Clean Data

## Drop Unnecessary Columns 

In [ ]:
df = csv.drop(columns=['itemType','collection','public','featured'])

## Add Image Column

In [ ]:
df['image'] = df['file']

In [ ]:
df.head()

## Change Hostnames in Columns

In [ ]:
image_path = 'https://newserver.org/sse/' + str(year) + '/images/'
pdf_path = 'https://newserver.org/sse/' + str(year) + '/pdf/'

In [ ]:
df['file'].replace(to_replace='http://oldserver.org/sse/', inplace=True, regex=True, value=pdf_path)

In [ ]:
df['image'].replace(to_replace='http://oldserver.org/sse/', inplace=True, regex=True, value=image_path)

In [ ]:
df['Item Type Metadata:Filename'].replace(to_replace='http://oldserver.org/sse/', inplace=True, regex=True, value='')

In [ ]:
df.head()

## Remove Punctuation From Filenames

In [ ]:
def hrefReplace(f):
    dirname = os.path.dirname(f) + "/"
    fname = os.path.basename(f)
    name,ext=os.path.splitext(fname)
    name = name.replace('.','_',1)
    name += ext
    f = dirname + name
    return f

In [ ]:
df['Item Type Metadata:Filename'] = df['Item Type Metadata:Filename'].apply(hrefReplace)

In [ ]:
df['file'] = df['file'].apply(hrefReplace)

In [ ]:
df['image'] = df['image'].apply(hrefReplace)

In [ ]:
df['image'][0]

In [ ]:
df.head()

## Previous Step Added A Slash 

In [ ]:
df['Item Type Metadata:Filename'].replace(to_replace='/', inplace=True, regex=True, value='')

## Add iframe Syntax to File Column

In [ ]:
df['file'] = df['file'].apply(lambda x: f'<iframe width="100%" height="500px" src="{x}"></iframe>')

## Replace File Extensions in Image Column

In [ ]:
df['image'].replace(to_replace='.pdf', inplace=True, regex=True, value='.jpg')

In [ ]:
df['image'][0]

## Rename File Column
To save time, we need to change the 'file' column to read 'Dublin Core:Source' so it'll be automatically assigned in the batch upload process.

In [ ]:
df.rename(columns={'file': 'Dublin Core:Source'}, inplace=True)

## Write to New File

In [ ]:
import csv
export_csv=str(year) + '_SSE_df.csv'

df.to_csv(export_csv, 
          sep=";", 
          date_format="%Y", 
          index=False, 
          quotechar="",  
          escapechar="\\",
          quoting=csv.QUOTE_NONE)